In [ ]:
from operator import itemgetter
import matplotlib.pyplot as plt
import firedrake
from firedrake import inner, sym, tr, grad, Constant, dx, ds
import icepack
from icepack.constants import (
    weertman_sliding_law,
    glen_flow_law,
    gravity as g,
    ice_density as ρ_I,
    water_density as ρ_W,
)

In [ ]:
Lx, Ly = 50e3, 12e3
nx, ny = 48, 32
mesh = firedrake.RectangleMesh(nx, ny, Lx, Ly)

Q = firedrake.FunctionSpace(mesh, "CG", 2)
V = firedrake.VectorFunctionSpace(mesh, "CG", 2)

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)

b_in, b_out = 200, -400
b = firedrake.Function(Q).interpolate(b_in - (b_in - b_out) * x / Lx)

s_in, s_out = 850, 50
s0 = firedrake.Function(Q).interpolate(s_in - (s_in - s_out) * x / Lx)

h0 = firedrake.Function(Q).interpolate(s0 - b)

s = s0.copy(deepcopy=True)
h = h0.copy(deepcopy=True)

In [ ]:
h_in = s_in - b_in
δs_δx = (s_out - s_in) / Lx
τ_D = -ρ_I * g * h_in * δs_δx
print(f"{1000 * τ_D} kPa")

In [ ]:
u_in, u_out = 20, 2400
velocity_x = u_in + (u_out - u_in) * (x / Lx) ** 2
u0 = firedrake.Function(V).interpolate(firedrake.as_vector((velocity_x, 0)))
u = u0.copy(deepcopy=True)

In [ ]:
T = firedrake.Constant(255.0)
A = icepack.rate_factor(T)

In [ ]:
m = Constant(weertman_sliding_law)
expr = (0.95 - 0.05 * x / Lx) * τ_D / u_in**(1 / m)
C = firedrake.Function(Q).interpolate(expr)

p_W = ρ_W * g * firedrake.max_value(0, h0 - s0)
p_I = ρ_I * g * h0
ϕ = 1 - p_W / p_I

In [ ]:
n = Constant(glen_flow_law)

def friction(**kwargs):
    names = ("velocity", "thickness", "surface", "friction")
    u, h, s, C = itemgetter(*names)(kwargs)

    p_W = ρ_W * g * firedrake.max_value(0, h - s)
    p_I = ρ_I * g * h
    ϕ = 1 - p_W / p_I
    return m / (m + 1) * C * ϕ * inner(u, u) ** ((m + 1) / (2 * m))

def viscosity(**kwargs):
    names = ("velocity", "thickness", "fluidity")
    u, h, A = itemgetter(*names)(kwargs)
    ε = sym(grad(u))
    εCε = (inner(ε, ε) + tr(ε)**2) / 2
    return 2 * n / (n + 1) * h * A ** (-1 / n) * εCε ** ((n + 1) / (2 * n))

def gravity(**kwargs):
    names = ("thickness", "surface", "velocity")
    h, s, u = itemgetter(*names)(kwargs)
    return -ρ_I * g * h * inner(grad(s), u)

def terminus(**kwargs):
    names = ("velocity", "thickness", "surface")
    u, h, s = itemgetter(*names)(kwargs)

    d = firedrake.min_value(s - h, 0)
    τ_I = ρ_I * g * h**2 / 2
    τ_W = ρ_W * g * d**2 / 2

    ν = firedrake.FacetNormal(u.function_space().mesh())
    return (τ_I - τ_W) * inner(u, ν)

In [ ]:
inflow_ids = (1,)
terminus_ids = (2,)
side_wall_ids = (3, 4)

fields = {"velocity": u, "thickness": h, "surface": s}
parameters = {"fluidity": A, "friction": C}
G = (
    viscosity(**fields, **parameters) * dx +
    friction(**fields, **parameters) * dx -
    gravity(**fields, **parameters) * dx -
    terminus(**fields, **parameters) * ds(terminus_ids)
)

F = firedrake.derivative(G, u)

In [ ]:
inflow_bc = firedrake.DirichletBC(V, u0, inflow_ids)
side_bcs = firedrake.DirichletBC(V.sub(1), 0, side_wall_ids)
bcs = [inflow_bc, side_bcs]

sparams = {"snes_linesearch_type": "nleqerr", "snes_monitor": None}
firedrake.solve(F == 0, u, bcs=bcs, solver_parameters=sparams)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_axis_off()
arrows = firedrake.quiver(u, axes=ax)
fig.colorbar(arrows, orientation="horizontal");